In [43]:
import requests
import pandas as pd

# Global Request Functions

In [44]:
header = {"X-Riot-Token": "RGAPI-7e521a6d-79b3-4b7c-bfad-3c6adcd287c2"}


def getSummoner(summoner):
    summoner = requests.get(
        f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}",
        headers=header,
    )
    return summoner.json()


def getMatchIDs(puuid):
    matchIDs = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100",
        headers=header,
    )
    return matchIDs.json()


def getMatch(matchID):
    matches = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}",
        headers=header,
    )
    return matches.json()

In [45]:
champions = {}

In [46]:

class Summoner:
    def __init__(self, summonerName):
        self.name = summonerName
        self.puuid = getSummoner(summonerName)["puuid"]
        self.matchIDs = getMatchIDs(self.puuid)
        self.partnerName = ""
        self.partnerPuuid = ""
        self.df = pd.DataFrame(columns=["MatchID", "ChampionID", "VisionScore", "KDA",
                           "PartnerChampionID", "PartnerVisionScore", "PartnerKDA", "Win/Lost"])

    def initPartner(self, partnerName):
        r = getSummoner(partnerName)
        self.partnerPuuid = r["puuid"]
        self.partnerName = r["name"]

        for i, matchID in enumerate(self.matchIDs):
            matchJSON = getMatch(matchID)


            try:
                if matchJSON["info"]["gameMode"] == "ARAM":
                    continue
 
                participants = matchJSON["info"]["participants"]
            except:
                break

            if self.partnerPuuid in matchJSON["metadata"]["participants"]:
                myNdx = 0
                partnerNdx = 0

                while participants[myNdx]["summonerName"] != self.name:
                    myNdx += 1
                while participants[partnerNdx]["summonerName"] != self.partnerName:
                    partnerNdx += 1

                me = participants[myNdx]
                partner = participants[partnerNdx]

                kda = round(participants[myNdx]["challenges"]["kda"], 2)
                partnerKDA = round(participants[partnerNdx]["challenges"]["kda"],  2)

                visionScore = round(participants[myNdx]["visionScore"], 2)
                partnerVisionScore = round(participants[partnerNdx]["visionScore"], 2)

                myChampion = me["championName"]
                partnersChampion = partner["championName"]

                if champions.get(myChampion) == None:
                    champions[myChampion] = me["championId"]
                if champions.get(myChampion) == None:
                    champions[myChampion] = partner["championId"]

                if me["win"]:
                    win = 1
                else:
                    win = 0


                newMatch = pd.Series(
                    [
                        matchID, myChampion, visionScore ,kda, partnersChampion, partnerVisionScore, partnerKDA, win
                    ],
                    index=["MatchID", "ChampionID", "VisionScore", "KDA",
                           "PartnerChampionID", "PartnerVisionScore", "PartnerKDA", "Win/Lost"],
                )

                print(f"Adding: {[matchID, myChampion, visionScore, kda, partnersChampion, partnerVisionScore, partnerKDA, win]} to DF")
                self.df.loc[i] = newMatch

In [47]:
mainSummoner = Summoner("41619")
mainSummoner.initPartner("yongbin")

Adding: ['NA1_4288895517', 'Rakan', 84, 5, 'Vayne', 13, 4.57, 1] to DF
Adding: ['NA1_4288901290', 'Pyke', 52, 1.6, 'Jinx', 8, 3, 0] to DF
Adding: ['NA1_4288837395', 'Alistar', 44, 3.5, 'Jinx', 8, 15, 1] to DF
Adding: ['NA1_4288841120', 'Renata', 64, 1.67, 'Vayne', 15, 2.2, 0] to DF
Adding: ['NA1_4288748320', 'Pyke', 13, 3.33, 'Jhin', 7, 10, 1] to DF
Adding: ['NA1_4288791665', 'Rakan', 31, 1.6, 'Zeri', 6, 0.29, 0] to DF
Adding: ['NA1_4288051179', 'Rakan', 61, 13.5, 'Jinx', 32, 25, 1] to DF
Adding: ['NA1_4287985938', 'Xerath', 64, 2.29, 'Vayne', 12, 1.78, 0] to DF
Adding: ['NA1_4287970218', 'Thresh', 82, 2.12, 'Jhin', 31, 2.44, 0] to DF
Adding: ['NA1_4287932752', 'Morgana', 78, 2, 'Caitlyn', 16, 2.33, 0] to DF
Adding: ['NA1_4283931327', 'Pyke', 82, 3.12, 'Kaisa', 28, 5.8, 1] to DF
Adding: ['NA1_4283835083', 'Senna', 46, 5.8, 'Jhin', 5, 4.25, 1] to DF
Adding: ['NA1_4283791260', 'Lux', 63, 4.5, 'Vayne', 18, 20, 1] to DF
Adding: ['NA1_4283674089', 'Bard', 79, 6.25, 'Ezreal', 9, 4, 1] to DF


KeyError: 'info'

In [ ]:
mainSummoner.df.to_csv('41binMatches.csv')


In [ ]:
print(champions)